In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [2]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large")

##### interact

In [3]:
# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User:hello
DialoGPT: Hiya
>> User:How are you?
DialoGPT: I'm good, you?
>> User:Fine. What you do today?
DialoGPT: I'm doing nothing.
>> User:Do you wish walk with me?
DialoGPT: I wish walk with you.
>> User:Ok lets go
DialoGPT: I'll be right there.


#### one line

In [35]:
user_text = "What's your best film?"
# encode the new user input, add the eos_token and return a tensor in Pytorch
new_user_input_ids = tokenizer.encode(user_text + tokenizer.eos_token, return_tensors='pt')
# append the new user input tokens to the chat history
bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
# generated a response while limiting the total chat history to 1000 tokens, 
chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
# pretty print last ouput tokens from bot
print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

DialoGPT: I love you.


#### multiple lines

In [53]:
chat_history = "|0|1|What weather do you like?|1|1|Sunny.|0|1|Do you ready for walk?"
bot_input_ids = tokenizer.encode(chat_history + tokenizer.eos_token, return_tensors='pt')
# generated a response while limiting the total chat history to 1000 tokens, 
chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
# pretty print last ouput tokens from bot
print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

DialoGPT: I'm ready for walk.


#### updated chat history

In [59]:
updated_chat_history = chat_history + '|1|1|' + tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
updated_chat_history

"|0|1|What weather do you like?|1|1|Sunny.|0|1|Do you ready for walk?|1|1|I'm ready for walk."